<a href="https://colab.research.google.com/github/sheikmohdimran/Experiments_2020/blob/master/NLP/GRC_POS_Tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#POS Tag Prediction
Since the input data was provided without tags, we will be using a freely available industry dataset, to train a POS tagger. The best model from the training will be used for predict tags for random text from the input file.

If this process has to be extended to train the provided dataset, the input sentences have to formatted to include the POS tags and in the existing format to be fed into the network.

For this solution I use Flair, a popular NLP library which has given outstanding results in various benchmarks.

In [1]:
!pip -q install flair==0.4.3

     |████████████████████████████████| 184kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 43.1MB/s 
     |████████████████████████████████| 778kB 40.4MB/s 
     |████████████████████████████████| 798kB 47.1MB/s 
     |████████████████████████████████| 184kB 44.5MB/s 
     |████████████████████████████████| 348kB 52.5MB/s 
     |████████████████████████████████| 1.0MB 42.6MB/s 
     |████████████████████████████████| 870kB 48.2MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.10 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.6.1 which is incompatible.


##Use Pre-trained Model for POS prediction

In [21]:
from flair.models import SequenceTagger
from flair.data import Sentence

tagger = SequenceTagger.load('pos')
sentence = Sentence('type iv securities. a national bank may purchase and sell type iv securities for its own account. the amount of the type iv securities that a bank may purchase and sell is not limited to a specified percentage of the banks capital and surplus.')
tagger.predict(sentence)
print(sentence.to_tagged_string())


2020-01-07 12:16:34,206 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/POS-ontonotes--h256-l1-b32-p3-0.5-%2Bglove%2Bnews-forward%2Bnews-backward-normal-locked0.5-word0.05--v0.4_0/en-pos-ontonotes-v0.4.pt not found in cache, downloading to /tmp/tmppopft3jb


100%|██████████| 432218302/432218302 [00:47<00:00, 9166621.05B/s]

2020-01-07 12:17:22,534 copying /tmp/tmppopft3jb to cache at /root/.flair/models/en-pos-ontonotes-v0.4.pt


2020-01-07 12:17:24,541 removing temp file /tmp/tmppopft3jb
2020-01-07 12:17:24,612 loading file /root/.flair/models/en-pos-ontonotes-v0.4.pt
type <NOUN> iv <NOUN> securities. <NOUN> a <DET> national <ADJ> bank <NOUN> may <AUX> purchase <VERB> and <CCONJ> sell <VERB> type <NOUN> iv <NOUN> securities <NOUN> for <ADP> its <PRON> own <ADJ> account. <NOUN> the <DET> amount <NOUN> of <ADP> the <DET> type <NOUN> iv <NOUN> securities <NOUN> that <PRON> a <DET> bank <NOUN> may <AUX> purchase <VERB> and <CCONJ> sell <VERB> is <VERB> not <ADV> limited <VERB> to <ADP> a <DET> specified <VERB> percentage <NOUN> of <ADP> the <DET> banks <NOUN> capital <NOUN> and <CCONJ> surplus. <NOUN>


##Train Custom Neural Net for POS prediction

In [0]:
from flair.data import Corpus
from flair.embeddings import TokenEmbeddings, StackedEmbeddings, FlairEmbeddings
from typing import List
from flair.datasets import WIKINER_ENGLISH

In [8]:
corpus: Corpus = WIKINER_ENGLISH()
print(corpus)

2020-01-07 09:38:41,620 Reading data from /root/.flair/datasets/wikiner_english
2020-01-07 09:38:41,622 Train: /root/.flair/datasets/wikiner_english/aij-wikiner-en-wp3.train
2020-01-07 09:38:41,624 Dev: None
2020-01-07 09:38:41,626 Test: None
Corpus: 115144 train + 12794 dev + 14215 test sentences


Input corpus is split into train(115144), dev(12794) and test(14215) sentences.

In [9]:
tag_type = 'pos'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

[b'<unk>', b'O', b'NNP', b'VBZ', b'VBN', b'DT', b'JJ', b'CC', b'NN', b'IN', b',', b'.', b'RB', b'PRP', b'NNS', b'PRP$', b'LRB', b'CD', b'RRB', b'VBG', b'TO', b'WDT', b'VBD', b':', b'NNPS', b'POS', b'VBP', b'LQU', b'VB', b'MD', b';', b'RP', b'RBR', b'WP', b'RQU', b'JJR', b'WP$', b'WRB', b'PDT', b'EX', b'RBS', b'JJS', b'AS', b'#', b'FW', b'$', b'UH', b'SO', b'SYM', b'LS', b'<START>', b'<STOP>']


In total the dataset is tagged with 52 POS Tags

In [12]:
embedding_types: List[TokenEmbeddings] = [
     FlairEmbeddings('news-forward'),
     FlairEmbeddings('news-backward'),
]

2020-01-07 09:44:15,974 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/big-news-forward--h2048-l1-d0.05-lr30-0.25-20/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmpqbytw1bn


100%|██████████| 73034624/73034624 [00:08<00:00, 8433465.93B/s] 

2020-01-07 09:44:25,767 copying /tmp/tmpqbytw1bn to cache at /root/.flair/embeddings/news-forward-0.4.1.pt


2020-01-07 09:44:25,861 removing temp file /tmp/tmpqbytw1bn
2020-01-07 09:44:37,560 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/big-news-backward--h2048-l1-d0.05-lr30-0.25-20/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmp2oyqlr3_


100%|██████████| 73034575/73034575 [00:08<00:00, 8165219.55B/s] 

2020-01-07 09:44:47,645 copying /tmp/tmp2oyqlr3_ to cache at /root/.flair/embeddings/news-backward-0.4.1.pt


2020-01-07 09:44:47,737 removing temp file /tmp/tmp2oyqlr3_


In [0]:
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

In [0]:
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)


In [0]:
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)


In [16]:
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)


2020-01-07 09:45:19,751 ----------------------------------------------------------------------------------------------------
2020-01-07 09:45:19,754 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4096, out_features=4096, bias=True)
  (rnn): LSTM(4096, 256, bidirectional=True)
  (linear): Linear(in_features=512, out_feature

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LanguageModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr

2020-01-07 10:20:37,231 ----------------------------------------------------------------------------------------------------
2020-01-07 10:20:38,819 epoch 2 - iter 0/3599 - loss 3.92844296 - samples/sec: 24713.02
2020-01-07 10:23:45,093 epoch 2 - iter 359/3599 - loss 3.70915679 - samples/sec: 64.00
2020-01-07 10:26:52,353 epoch 2 - iter 718/3599 - loss 3.69708515 - samples/sec: 64.02
2020-01-07 10:29:58,853 epoch 2 - iter 1077/3599 - loss 3.66007391 - samples/sec: 64.06
2020-01-07 10:33:04,451 epoch 2 - iter 1436/3599 - loss 3.61489447 - samples/sec: 64.39
2020-01-07 10:36:10,239 epoch 2 - iter 1795/3599 - loss 3.59747536 - samples/sec: 64.21
2020-01-07 10:39:16,614 epoch 2 - iter 2154/3599 - loss 3.55305630 - samples/sec: 64.02
2020-01-07 10:42:21,628 epoch 2 - iter 2513/3599 - loss 3.52778156 - samples/sec: 64.53
2020-01-07 10:45:27,853 epoch 2 - iter 2872/3599 - loss 3.50918568 - samples/sec: 64.14
2020-01-07 10:48:33,819 epoch 2 - iter 3231/3599 - loss 3.49426832 - samples/sec: 64.

Process Process-50:


2020-01-07 12:11:33,549 ----------------------------------------------------------------------------------------------------
2020-01-07 12:11:33,551 Exiting from training early.


Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()


2020-01-07 12:11:33,555 Saving model ...


  File "/usr/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 262, in _run_finalizers
    finalizer()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 191, in _finalize_join
    thread.join()
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7efd4bb563c8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 124, in join
    res = self._popen.wait(timeout)
  Fil

2020-01-07 12:11:34,014 Done.
2020-01-07 12:11:34,019 ----------------------------------------------------------------------------------------------------
2020-01-07 12:11:34,022 Testing using best model ...
2020-01-07 12:11:34,027 loading file resources/taggers/example-ner/best-model.pt
2020-01-07 12:15:32,686 0.9753	0.9753	0.9753
2020-01-07 12:15:32,688 
MICRO_AVG: acc 0.9518 - f1-score 0.9753
MACRO_AVG: acc 0.8645 - f1-score 0.9041106382978724
#          tp: 27 - fp: 1 - fn: 3 - tn: 27 - precision: 0.9643 - recall: 0.9000 - accuracy: 0.8710 - f1-score: 0.9310
$          tp: 126 - fp: 8 - fn: 0 - tn: 126 - precision: 0.9403 - recall: 1.0000 - accuracy: 0.9403 - f1-score: 0.9692
,          tp: 18314 - fp: 0 - fn: 0 - tn: 18314 - precision: 1.0000 - recall: 1.0000 - accuracy: 1.0000 - f1-score: 1.0000
.          tp: 13982 - fp: 0 - fn: 0 - tn: 13982 - precision: 1.0000 - recall: 1.0000 - accuracy: 1.0000 - f1-score: 1.0000
:          tp: 749 - fp: 95 - fn: 25 - tn: 749 - precision: 0.8

{'dev_loss_history': [tensor(2.2084, device='cuda:0'),
  tensor(1.9426, device='cuda:0'),
  tensor(1.8504, device='cuda:0'),
  tensor(1.7589, device='cuda:0')],
 'dev_score_history': [0.9704, 0.9729, 0.9742, 0.9749],
 'test_score': 0.9753,
 'train_loss_history': [6.1171543689263,
  3.4727155129821408,
  3.091912110966754,
  2.903443764533954]}

Fetching best model after 3 epochs

In [17]:
model = SequenceTagger.load('resources/taggers/example-ner/best-model.pt')

2020-01-07 12:15:49,592 loading file resources/taggers/example-ner/best-model.pt


Run prediction on random sentence.

In [19]:
sentence = Sentence('type iv securities. a national bank may purchase and sell type iv securities for its own account. the amount of the type iv securities that a bank may purchase and sell is not limited to a specified percentage of the banks capital and surplus.')
model.predict(sentence)
print(sentence.to_tagged_string())

type <NN> iv <NN> securities. <NN> a <DT> national <JJ> bank <NN> may <MD> purchase <VB> and <CC> sell <VB> type <NN> iv <NN> securities <NNS> for <IN> its <PRP$> own <JJ> account. <NN> the <DT> amount <NN> of <IN> the <DT> type <NN> iv <NN> securities <NNS> that <IN> a <DT> bank <NN> may <MD> purchase <VB> and <CC> sell <VB> is <VBZ> not <RB> limited <VBN> to <TO> a <DT> specified <JJ> percentage <NN> of <IN> the <DT> banks <NNS> capital <NN> and <CC> surplus. <NN>
